In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel agent. No follow up questions."
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct flight from San Francisco to Tokyo on March 31st")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to Tokyo on March 31st', additional_kwargs={}, response_metadata={}, id='716f68fd-84eb-47a0-b0b8-d5d2edd9993f'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to get a direct flight from SFO to Tokyo on March 31. Use search-flight. Provide cabin class default economy, currency default EUR maybe. Provide departure date 31/03/2026 (assuming year). The user didn\'t specify year, but today is 2026-02-09, so likely 31/03/2026. Provide flyFrom "San Francisco" (SFO) and flyTo "Tokyo". Need direct flight only. The API may have a parameter for max stopovers? Not in spec. But we can filter results after retrieving. We\'ll request and then filter for routes with only one segment (no layovers). The tool returns flights; we need to display table grouped by cheapest, shortest, rest. We\'ll need to process results. Let\'s call search-flight.', 'tool_calls': [{'id': 'fc_cf96b9a8-5c51-4c

In [7]:
print(response["messages"][-1].content)

**Direct flights San Francisco (SFO) → Tokyo (NRT)**  

| Route | Departure → Arrival (local) & Duration | Cabin | Price (USD) | Book |
|-------|----------------------------------------|-------|-------------|------|
| **San Francisco SFO → Tokyo NRT** | 31/03 16:45 → 01/04 20:00 (11 h 15 m) | Economy | **$677** | [Link](https://on.kiwi.com/jMfDBI) |
| **San Francisco SFO → Tokyo NRT** | 31/03 12:20 → 01/04 15:25 (11 h 5 m) | Economy | $1 262 | [Link](https://on.kiwi.com/a5Roy5) |

### Summary & Recommendation
- **Best price:** the $677 flight (11 h 15 m) is the cheapest direct option.  
- **Shortest duration:** the $1 262 flight (11 h 5 m) is a bit faster, though it costs more.  

If you prefer to save money and the extra 10 minutes isn’t a deal‑breaker, the $677 flight is the clear choice. Otherwise, the slightly quicker $1 262 option may be worth the premium.

**Have a wonderful trip to Tokyo!**  
*Fun fact:* Tokyo’s Shibuya Crossing is said to be the world’s busiest pedestrian scram